In [1]:
!pip install transformers torch

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -------------- ------------------------- 4.2/11.6 MB 21.1 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.6 MB 26.2 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 24.4 MB/s  0:00:00
   ---------------------------------------- 0.0/561.5 kB ? eta -:--:--
   ---------------------------------------- 561.5/561.5 kB 36.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 24.5 MB/s  0:00:00

   ---------------------------------------- 0/5 [safetensors]
   -------- ------------------------------- 1/5 [regex]
   ---------------- ----------------------- 2/5 [huggingface-hub]
   ---------------- ----------------------- 2/5 [huggingface-hub]
   ---------------- ----------------------- 2/5 [huggingface-hub]
   ---------------- ----------------------- 2/5 [huggingface-hub]
   ---------------- --

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. 모델과 토크나이저 불러오기
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# GPU 사용 가능 시 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. 대화 함수 정의
def chat():
    print("=== 한국어 GPT-2 대화형 챗봇 (개선 버전) ===")
    print("종료하려면 'quit' 입력\n")

    chat_history_ids = None # 토큰화된 대화 기록 저장용
    max_history_tokens = 512 # 최대 대화 기록 토큰 수

    while True:
        user_input = input("User: ")
        if user_input.lower() == "quit":
            print("Bot: 대화를 종료합니다. 안녕히 가세요!")
            break

        # 사용자의 입력 토큰화
        new_input_ids = tokenizer(f"User: {user_input}\nBot:", return_tensors="pt").input_ids.to(device)

        # 기존 대화 기록이 있으면 합치기
        if chat_history_ids is None:
            chat_history_ids = new_input_ids
        else:
            # 대화 기록 길이 제한 (최대 512 토큰)
            # 새로운 입력을 추가했을 때 512를 넘지 않도록, 오래된 기록을 자름
            total_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
            if total_ids.shape[-1] > max_history_tokens:
                chat_history_ids = total_ids[:, -max_history_tokens:] # 최근 512개 토큰만 남김
            else:
                chat_history_ids = total_ids

        # 답변 생성
        outputs = model.generate(
            input_ids=chat_history_ids, # chat_history_ids는 대화의 이전 내용(프롬프트)을 이미 토큰 ID로 변환한 리스트(텐서)이며, 모델은 이 시퀀스에 이어서 다음 단어를 생성
            max_length=len(chat_history_ids[0]) + 100, # 답변 길이를 동적으로 설정
            pad_token_id=tokenizer.eos_token_id, # 길이를 맞추기 위한 패딩 토큰
            do_sample=True, # 모델의 출력 토큰을 선택할 때 샘플링(Sampling) 방식을 사용할지 여부
                          # True로 설정하면 모델이 예측한 확률 분포를 바탕으로 **무작위성(랜덤성)**을 도입하여 토큰을 선택. 
                          # False일 경우 항상 가장 확률이 높은 토큰(Greedy Search)을 선택
            top_p=0.9,   # 샘플링 방법 중 하나인 Top-P (Nucleus Sampling)의 임계값. 
                         # 확률이 매우 낮은 이상하거나 무의미한 단어가 선택되는 것을 방지하면서도 일정 수준의 다양성을 유지
            temperature=0.8, # 모델의 예측 확률 분포를 조정하는 매개변수(온도) 
                             # Temperature 값	확률 분포의 변화	출력 특성 (일관성 vs. 다양성)
                             # 높을수록 :	평탄해짐	다양성, 창의성 증가 (일관성 감소)
                             # 낮을수록 :	뾰족해짐	일관성, 보수성 증가 (다양성 감소)
                             # 0에 가까울수록 :	가장 뾰족함	가장 일관적 (항상 가장 확률 높은 토큰만 선택)
                             # 1.0 :	조정 없음	모델이 계산한 원래 확률 분포 그대로 사용
            repetition_penalty=1.2 # 모델이 이전에 이미 생성한 토큰을 다시 선택할 확률을 감소시키는 패널티 값
                            # 값이 1.0보다 크면 (예: 1.2) 반복을 억제하고, 값이 1.0보다 작으면 반복을 장려
        )

        # 결과 디코딩 및 추출
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        bot_reply = response.split("Bot:")[-1].split("User:")[0].strip()

        print(f"Bot: {bot_reply}")

        # 대화 기록 업데이트 (다음 턴을 위해)
        # 생성된 답변까지 포함하여 다음 입력으로 사용
        chat_history_ids = tokenizer(response, return_tensors="pt").input_ids.to(device)
        # 변환된 토큰 시퀀스를 PyTorch 텐서(Tensor) 형태로 반환하도록 지정합니다. (pt는 PyTorch를 의미

# 3. 챗봇 실행
if __name__ == "__main__":
    chat()